# # # # This script gathers all information from Dataverse, datasets and files API, and only gets NEW ADDED UPLOADS. The script puts all in the same column to be analysed.

In [2]:
## Beforehand, it must be downloaded py_d3 (pip install py_d3 in console)
%load_ext py_d3

In [3]:
import calendar
import requests
import json
import pandas as pd
import datetime

req = requests.Session()
files=[]
dataverses=[]
datasets=[]
month=[]
datetoday=datetime.datetime.today()
urlbase='https://dataverse.harvard.edu/api/info/metrics/'


# Defining a function to analyse certain months, based on today's date.
def change_months(datetoday, months):
     month = datetoday.month - 1 + months
     year = datetoday.year + month // 12
     month = month % 12 + 1
     day = min(datetoday.day,calendar.monthrange(year,month)[1])
     return datetime.date(year,month,day)
    
from datetime import datetime
from dateutil import relativedelta

date1 = datetime.strptime('2007-11-01 12:00:00', '%Y-%m-%d %H:%M:%S') ## This is the month we consider Harvard Database creation, as it is the first month we have uploads.
date2 = datetime.strptime(datetoday.strftime('%Y-%m-%d %H:%M:%S'), '%Y-%m-%d %H:%M:%S')
r = relativedelta.relativedelta(date2, date1)
monthsfromdataversecreation=(r.years)*12+r.months
print("Months since Dataverse Creation: "+str(monthsfromdataversecreation))


import datetime
for i in range (1, monthsfromdataversecreation+1):
    # Get  today's month
    month.append(str(change_months(datetoday,-monthsfromdataversecreation+i-1).strftime("%Y-%m")))
    
    # Get evolution of number of files uploaded since the beginning of the (data)universe, by jumping on each website.
    url=urlbase+"files/toMonth/"+str(change_months(datetoday,-monthsfromdataversecreation+i-1).strftime("%Y-%m"))
    data = json.loads(req.get(url).text)
    if i == 1:
        files.append(0)
        keepvalue=data['data']['count']
    if i > 1:
        files.append(data['data']['count']-keepvalue)
        keepvalue=data['data']['count']
    
    # Get evolution of number of dataverses since the beginning of the (data)universe
    url=urlbase+"dataverses/toMonth/"+str(change_months(datetoday,-monthsfromdataversecreation+i-1).strftime("%Y-%m"))
    data = json.loads(req.get(url).text)
    if i == 1:
        dataverses.append(0)
        keepvalue2=data['data']['count']
    if i > 1:
        dataverses.append(data['data']['count']-keepvalue2)
        keepvalue2=[]
        keepvalue2=data['data']['count']

    # Get evolution of number of datasets since the beginning of the (data)universe
    url=urlbase+"datasets/toMonth/"+str(change_months(datetoday,-monthsfromdataversecreation+i-1).strftime("%Y-%m"))
    data = json.loads(req.get(url).text)
    if i == 1:
        datasets.append(0)
        keepvalue3=data['data']['count']
    if i > 1:
        datasets.append(data['data']['count']-keepvalue3)
        keepvalue3=data['data']['count']
    
    
# # CREATING DATABASE: RELATIONSHIP BETWEEN NUMBER OF DATAVERSEs, DATASETS AND FILES. First, we add the column origin's name.

months=pd.DataFrame(month, columns=['Month'])

files_df=pd.DataFrame(files, columns=['UploadsNumber'])
files_df['Type']="Files"
files_df_final=pd.merge(months,files_df, left_index=True,right_index=True)

dataverses_df=pd.DataFrame(dataverses, columns=['UploadsNumber'])
dataverses_df['Type']="Dataverse"
dataverses_df_final=pd.merge(months,dataverses_df, left_index=True,right_index=True)

datasets_df=pd.DataFrame(datasets, columns=['UploadsNumber'])
datasets_df['Type']="Datasets"
datasets_df_final=pd.merge(months,datasets_df, left_index=True,right_index=True)


addedtotals=pd.concat([files_df_final, dataverses_df_final, datasets_df_final])
addedtotals.head()


Months since Dataverse Creation: 134


,Month,UploadsNumber,Type
0,2007-11,0,Files
1,2007-12,539,Files
2,2008-01,450,Files
3,2008-02,768,Files
4,2008-03,57,Files


In [4]:
addedtotals.to_csv('added.csv', sep=";")
